In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/battles']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

#Game of Thrones
##The exploratory analysis of the battles

This is my first notebook on Kaggle and it has the objective of exploring the data available on Kaggle about the battles in the five first books of the collection A Song of Ice and Fire from George R. R. Martin.

###Importing the packages necessary

In [1]:
import numpy as np
# import pandas as pd
exec(os.environ['IREWR_IMPORTS'])
# ALEX: remove plotting
# import matplotlib.pyplot as plt

# %matplotlib inline

# plt.rcParams['figure.figsize'] = (10, 6)
# plt.style.use('ggplot')

###Loading and reading the data

In [2]:
battles = pd.read_csv('./input/battles.scaled.csv')

'''
there is an error on the data
on the Battle of Castle Black, it was Mance Rayder the attacker, not Stannis Baratheon
'''
# ALEX: make notebook run
# battles.set_value(27, 'attacker_king', 'Mance Rayder')
# battles.set_value(27, 'defender_king', 'Stannis Baratheon')
battles.at[27, 'attacker_king'] = 'Mance Rayder'
battles.at[27, 'defender_king'] = 'Stannis Baratheon'

print(battles.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                38 non-null     object 
 1   year                38 non-null     int64  
 2   battle_number       38 non-null     int64  
 3   attacker_king       36 non-null     object 
 4   defender_king       35 non-null     object 
 5   attacker_1          38 non-null     object 
 6   attacker_2          10 non-null     object 
 7   attacker_3          3 non-null      object 
 8   attacker_4          2 non-null      object 
 9   defender_1          37 non-null     object 
 10  defender_2          2 non-null      object 
 11  defender_3          0 non-null      float64
 12  defender_4          0 non-null      float64
 13  attacker_outcome    37 non-null     object 
 14  battle_type         37 non-null     object 
 15  major_death         37 non-null     float64
 16  major_capt

In [3]:
battles.head(5)

,name,year,battle_number,attacker_king,defender_king,attacker_1,attacker_2,attacker_3,attacker_4,defender_1,...,major_death,major_capture,attacker_size,defender_size,attacker_commander,defender_commander,summer,location,region,note
0,Battle of the Golden Tooth,298,1,Joffrey/Tommen Baratheon,Robb Stark,Lannister,NaN,NaN,NaN,Tully,...,1.0,0.0,15000.0,4000.0,Jaime Lannister,"Clement Piper, Vance",1.0,Golden Tooth,The Westerlands,NaN
1,Battle at the Mummer's Ford,298,2,Joffrey/Tommen Baratheon,Robb Stark,Lannister,NaN,NaN,NaN,Baratheon,...,1.0,0.0,NaN,120.0,Gregor Clegane,Beric Dondarrion,1.0,Mummer's Ford,The Riverlands,NaN
2,Battle of Riverrun,298,3,Joffrey/Tommen Baratheon,Robb Stark,Lannister,NaN,NaN,NaN,Tully,...,0.0,1.0,15000.0,10000.0,"Jaime Lannister, Andros Brax","Edmure Tully, Tytos Blackwood",1.0,Riverrun,The Riverlands,NaN
3,Battle of the Green Fork,298,4,Robb Stark,Joffrey/Tommen Baratheon,Stark,NaN,NaN,NaN,Lannister,...,1.0,1.0,18000.0,20000.0,"Roose Bolton, Wylis Manderly, Medger Cerwyn, H...","Tywin Lannister, Gregor Clegane, Kevan Lannist...",1.0,Green Fork,The Riverlands,NaN
4,Battle of the Whispering Wood,298,5,Robb Stark,Joffrey/Tommen Baratheon,Stark,Tully,NaN,NaN,Lannister,...,1.0,1.0,1875.0,6000.0,"Robb Stark, Brynden Tully",Jaime Lannister,1.0,Whispering Wood,The Riverlands,NaN


###Exploring the battles data

####Confrontations between kings

Number of confrontations between kings (attacker king, defender king)

In [4]:
# ALEX: remove plotting
# battles.groupby(['attacker_king', 'defender_king']).count()['name'].plot(kind = 'barh')
battles.groupby(['attacker_king', 'defender_king']).count()['name']

attacker_king             defender_king           
Balon/Euron Greyjoy       Balon/Euron Greyjoy          1
                          Joffrey/Tommen Baratheon     2
                          Robb Stark                   4
Joffrey/Tommen Baratheon  Balon/Euron Greyjoy          1
                          Robb Stark                  10
                          Stannis Baratheon            2
Mance Rayder              Stannis Baratheon            1
Robb Stark                Balon/Euron Greyjoy          1
                          Joffrey/Tommen Baratheon     9
Stannis Baratheon         Balon/Euron Greyjoy          1
                          Joffrey/Tommen Baratheon     2
                          Renly Baratheon              1
Name: name, dtype: int64

####Outcome of battles by king

In [5]:
# ALEX: remove plotting
# battles.groupby(['attacker_king','attacker_outcome']).count()['name'].unstack().plot(kind = 'barh')
battles.groupby(['attacker_king','attacker_outcome']).count()['name'].unstack()

attacker_outcome,loss,win
attacker_king,,
Balon/Euron Greyjoy,NaN,7.0
Joffrey/Tommen Baratheon,1.0,13.0
Mance Rayder,1.0,NaN
Robb Stark,2.0,8.0
Stannis Baratheon,1.0,2.0


####Commanders experience

Number of battles that each character has commanded as attacker or defender described in the books. For a better plot only the 15 first commanders in order of number of battles were selected for the plot.

In [6]:
def commander_battles(attack, defend):
    namesdict = {}
    for i in range(len(attack.index)):
        for name in attack.index[i].split(', '):
            if name in namesdict:
# ALEX: make notebook run
#                 namesdict[name] = namesdict[name] + attack.ix[i]
                namesdict[name] = namesdict[name] + attack.iloc[i]
            else:
# ALEX: make notebook run
#                 namesdict[name] = attack.ix[i]
                namesdict[name] = attack.iloc[i]
    for i in range(len(defend.index)):
        for name in defend.index[i].split(', '):
            if name in namesdict:
# ALEX: make notebook run
#                 namesdict[name] = namesdict[name] + defend.ix[i]
                namesdict[name] = namesdict[name] + defend.iloc[i]
            else:
# ALEX: make notebook run
#                 namesdict[name] = defend.ix[i]
                namesdict[name] = defend.iloc[i]
    return namesdict

commander_exp = commander_battles(battles.groupby('attacker_commander').count()['name'], battles.groupby('defender_commander').count()['name'])
commander_exp = pd.DataFrame.from_dict(commander_exp, 'index')
commander_exp.rename(columns = {0:'num_battles'}, inplace = True)
# ALEX: remove plotting
# commander_exp.sort_values('num_battles').tail(15).plot(kind = 'barh')
commander_exp.sort_values('num_battles').tail(15)

,num_battles
Theon Greyjoy,2
Rodrik Cassel,2
Addam Marbrand,2
Walder Frey,2
Asha Greyjoy,2
Tytos Blackwood,3
Randyll Tarly,3
Victarion Greyjoy,3
Tywin Lannister,3
Brynden Tully,4


####Type of battle preference

Number of the different types of battles that the attackers king participated.

In [7]:
# ALEX: remove plotting
# battles.groupby(['attacker_king', 'battle_type']).count()['name'].unstack().plot(kind = 'barh')
battles.groupby(['attacker_king', 'battle_type']).count()['name'].unstack()

battle_type,ambush,pitched battle,razing,siege
attacker_king,,,,
Balon/Euron Greyjoy,2.0,2.0,1.0,2.0
Joffrey/Tommen Baratheon,3.0,6.0,NaN,5.0
Mance Rayder,NaN,NaN,NaN,1.0
Robb Stark,5.0,3.0,NaN,2.0
Stannis Baratheon,NaN,2.0,NaN,1.0


####Size of armies and outcome for the attacker

In [8]:
# ALEX: remove plotting
# ax = battles[battles['attacker_outcome'] == 'win'].plot(kind = 'scatter', y = 'attacker_size', x = 'defender_size', label = 'win', s=100)
# battles[battles['attacker_outcome'] == 'loss'].plot(kind = 'scatter', y = 'attacker_size', x = 'defender_size', label = 'loss', color = 'r', s=100, ax = ax)
battles[battles['attacker_outcome'] == 'win']
battles[battles['attacker_outcome'] == 'loss']

,name,year,battle_number,attacker_king,defender_king,attacker_1,attacker_2,attacker_3,attacker_4,defender_1,...,major_death,major_capture,attacker_size,defender_size,attacker_commander,defender_commander,summer,location,region,note
3,Battle of the Green Fork,298,4,Robb Stark,Joffrey/Tommen Baratheon,Stark,NaN,NaN,NaN,Lannister,...,1.0,1.0,18000.0,20000.0,"Roose Bolton, Wylis Manderly, Medger Cerwyn, H...","Tywin Lannister, Gregor Clegane, Kevan Lannist...",1.0,Green Fork,The Riverlands,NaN
16,Battle of the Fords,299,17,Joffrey/Tommen Baratheon,Robb Stark,Lannister,NaN,NaN,NaN,Tully,...,0.0,0.0,20000.0,10000.0,"Tywin Lannister, Flement Brax, Gregor Clegane,...","Edmure Tully, Jason Mallister, Karyl Vance",1.0,Red Fork,The Riverlands,NaN
19,Battle of the Blackwater,299,20,Stannis Baratheon,Joffrey/Tommen Baratheon,Baratheon,NaN,NaN,NaN,Lannister,...,1.0,1.0,21000.0,7250.0,"Stannis Baratheon, Imry Florent, Guyard Morrig...","Tyrion Lannister, Jacelyn Bywater, Sandor Cleg...",1.0,King's Landing,The Crownlands,NaN
21,Battle of Duskendale,299,22,Robb Stark,Joffrey/Tommen Baratheon,Stark,NaN,NaN,NaN,Lannister,...,1.0,0.0,3000.0,NaN,"Robertt Glover, Helman Tallhart","Randyll Tarly, Gregor Clegane",1.0,Duskendale,The Crownlands,NaN
27,Battle of Castle Black,300,28,Mance Rayder,Stannis Baratheon,Free folk,Thenns,Giants,NaN,Night's Watch,...,1.0,1.0,100000.0,1240.0,"Mance Rayder, Tormund Giantsbane, Harma Dogshe...","Stannis Baratheon, Jon Snow, Donal Noye, Cotte...",0.0,Castle Black,Beyond the Wall,NaN


####Number of battles per region

In [9]:
# ALEX: remove plotting
# battles.groupby(['region']).count()['name'].sort_values().plot(kind = 'barh')
battles.groupby(['region']).count()['name'].sort_values()

region
Beyond the Wall     1
The Crownlands      2
The Reach           2
The Stormlands      3
The Westerlands     3
The North          10
The Riverlands     17
Name: name, dtype: int64